<h1>Spotted samples</h1>
<h3>Analyze peak intensity for spotted MSI data</h3>

In [ ]:
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ast
import requests, json
import Arrayed_Analysis_Tools as aa_tools
# plt.get_backend()
# plt.switch_backend('Qt4Agg')
client = requests.Session()

<h1>Authenticate to NERSC</h1>
<h3>Put your NERSC username in the quotes below and enter password in the box.  Requires up to date IPython to work</h3>

In [ ]:
aa_tools = reload(aa_tools)
username = raw_input('Enter your username: ')
client = aa_tools.authenticateUser(client,username) # put your username here in the quotes

##### Test that we are authenticated #####
##### by printing a list of files #####
##### you have access to #####
myFiles = aa_tools.getFilelist(client)
for k in myFiles:
    print k
##### If you don't see private files. #####
##### It didn't authenticate.  #####

<h1>Specify the full path to the file you want</h1>
<h3>You also have to specify the data and/or ana index</h3>
<h3>For multi-experiment files, you have to specify the experiment index</h3>

In [ ]:
filename = '/project/projectdirs/openmsi/omsi_data_private/bpb/20120913_nimzyme.h5'
# filename = '/project/projectdirs/openmsi/omsi_data_private/joshvh/20151104_GH1_G2.h5'
# filename = '/project/projectdirs/openmsi/omsi_data_private/raad0102/20150331MdR_MALDI-NIMS_1.h5'
dataIndex = '0'
expIndex = '0' ### for single experiment files, the experiment index is zero ###
payload = {'file':filename,'format':'JSON','mtype':'file','expIndex':expIndex,'dataIndex':dataIndex}
url = 'https://openmsi.nersc.gov/openmsi/qmetadata'
r = client.get(url,params=payload)
metadata = json.loads(r.content)
originalSize = ast.literal_eval(metadata[u'children'][0][u'shape'])
print originalSize
### prints the [x, y, m/z] of the data cube

<h1>Specify the ions for analysis</h1>

In [ ]:
myIons = [1241.25, 917.5, 817.6]
# myIons = [1102.15, 940, 1102]
# myIons = [2093.1, 2465.2, 2867.0, 3494.7, 3657.9, 5735.0]
myRange = 0.5 # integrate at +/- this amount

<h1>Get the images for each ion of interest</h1>
<h3>Change the "np.sum" to "np.max" for peak height instead of peak area</h3>

In [ ]:
mz = aa_tools.getMZ(client,filename,expIndex,dataIndex)
imStack = np.zeros((originalSize[0],originalSize[1],len(myIons)))
# for each ion in myions, get an image for that ion
for i,ion in enumerate(myIons):
    idx = np.where(abs(mz-ion)<myRange) #get the m/z indices within myRange
    payload = {'file':filename,
               'expIndex':expIndex,'dataIndex':dataIndex,'format':'JSON','mz':'%d:%d'%(min(idx[0]),max(idx[0]))}
    url = 'https://openmsi.nersc.gov/openmsi/qcube'
    r = client.get(url,params=payload)
    data = np.asarray(json.loads(r.content))
    ### to do: add background subtraction for "peakless" data ###
#     imStack[:,:,i] = np.sum(data,2) #### Pick this one for peak area
    imStack[:,:,i] = np.max(data,2) #### Pick this one for peak height

##### SELECT ONE OF THE METHODS BELOW FOR MAKING BASEIMAGE #####
##### baseImage is used to make the visualization for placing the markers #####
##### 1.  Here baseImage is the sum intensity of all your ion images #####
# baseImage = np.sum(imStack,2)  #ALL IONS

##### 2.  Here baseImage is a specific ion images #####
# baseImage = imStack[:,:,0] #ONE ION

##### baseImage is used to make the visualization for placing the markers #####
##### 3.  Here baseImage is the sum intensity of specific ion images #####
myIdx = [0,1,2]
baseImage = np.sum(imStack[:,:,myIdx],2) #SPECIFIC IONS
plt.imshow(baseImage,cmap='CMRmap')
plt.show()

Set the circles where they need to be.  Then close the figure window. The last coordinates of the circles will be stored and you use them to study the interior points of your trapezoid.

In [ ]:
Nrows = 24
Ncols = 16
dragRadius = 4
pointMarkerSize = 12
xCenter,yCenter = aa_tools.roughPosition(baseImage,Nrows,Ncols,dragRadius,pointMarkerSize)

In [ ]:
markerRadius = 2
xCenter,yCenter = aa_tools.fineTunePosition(baseImage,xCenter,yCenter,markerRadius)

In [ ]:
# Save a dictionary into a pickle file.

import pickle

spotset_data = {'xCenter':xCenter,'yCenter':yCenter,
            'Nrows':Nrows,'Ncols':Ncols,
            'dragRadius':dragRadius,'pointMarkerSize':pointMarkerSize,'markerRadius':markerRadius,
            'baseImage':baseImage,'imStack':imStack,'myIons':myIons,'myRange':myRange,'mz':mz,'originalSize':originalSize,
            'filename':filename,'expIndex':expIndex,'dataIndex':dataIndex}

pickle.dump( spotset_data, open( "myWorkspace.pkl", "wb" ) )

In [ ]:
# Load the dictionary back from the pickle file.
import pickle

spotset_data = pickle.load( open( "myWorkspace.pkl", "rb" ) )
for key in spotset_data:
    globals()[key]=spotset_data[key]

In [ ]:
xEdges, yEdges = np.meshgrid(range(imStack.shape[1]), range(imStack.shape[0]), sparse=False, indexing='xy')
mask = np.zeros(baseImage.shape)
integrationRadius = 6 #distance to integrate
myPixels = []
for x,y in zip(xCenter,yCenter):
    idx = np.argwhere(((x - xEdges)**2 + (y - yEdges)**2)**0.5 < integrationRadius)
    myPixels.append(idx)
    for i in idx:
        mask[i[0],i[1]] = 1
plt.imshow(mask)
plt.show()
    

In [ ]:
minIntensity = 100

In [ ]:
fid = open('export.tab','wb')
fid.write('index\tfile\trow\tcolumn\trow-centroid\tcol-centroid\t')
for i in myIons:
    fid.write('%5.4f Sum\t' % i)
    fid.write('%5.4f Max\t' % i)
    fid.write('%5.4f Mean\t' % i)
    fid.write('%5.4f Median\t' % i)
    fid.write('%5.4f NumPixels\t' % i)
fid.write('\n')

for i,myPixel in enumerate(myPixels): #how many spots
    fid.write('%d\t%s\t%s\t%s\t%d\t%d\t' % ( i, filename, 'coming_soon', 'coming_soon', np.mean(myPixel[:,0]), np.mean(myPixel[:,1]) ) )
    for i,ion in enumerate(myIons): #how many ions
        values = []
        for j, coord in enumerate(myPixel): #how many pixels per spot
            if imStack[coord[0],coord[1],i] > minIntensity:
                print imStack[coord[0],coord[1],i]
                print coord[0],coord[1],i
                #accumulate a list of peak height or 
                #peak area values for each pixel 
                #assigned to each spot
                values.append(imStack[coord[0],coord[1],i]) 
        if len(values) > 0:        
            fid.write('%d\t%d\t%d\t%d\t%d\t' % (np.sum(values),np.max(values),np.mean(values),np.median(values),len(values)))
        else:
            fid.write('%d\t%d\t%d\t%d\t%d\t' % (0,0,0,0,len(values)))
        
    fid.write('\n')
fid.close()

In [ ]:
myPixels